# Les variables binaires en optimisation 
Tentative de clarification et étude par estelle derrien

- Valeurs : Elles prennent la valeur 1 ou 0 (oui ou non).
- Intérêt : Sélectionner une variable de décision , Sélectionner une contrainte .
- Utilisation : Dans le fonction objectif, dans les contraintes.
- Déclaration : Elle diffère selon les solveurs.
- Semblable à : IF ( Si en Anglais) THEN
- Relaté à : Les méthodes "Big M"
- Permettent de : Eviter de faire une optimisation non linéaire.

# Exemples oraux :

Si les ventes de la variable de décision X1 dépassent 2000, alors la contrainte N s'applique.

